In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from ydata_profiling import ProfileReport
import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
df1 = pd.read_csv('data/daily_calendar_with_events.csv')
df2 = pd.read_csv('data/item_prices.csv')
df3 = pd.read_csv('data/item_sales.csv')

In [3]:
df1.info()
print("----------")
df2.info()
print("----------")
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [ ]:
df1.head(3)

In [ ]:
df1['date'] = pd.to_datetime(df1.date)
df1['event'] = df1['event'].fillna('Normalday')

In [ ]:
df1["weekyear"] = df1['date'].dt.isocalendar().week
df1["year"] = df1['date'].dt.year

In [ ]:
def g(x):
    return str(x[0]) + str(x[1])+".0"

df1['yearweek'] = df1[['year', 'weekyear']].apply(g, axis=1)

In [ ]:
df1['yearweek'] = df1.yearweek.astype('float64')

In [ ]:
df1['event'].value_counts()

In [ ]:
df1.groupby(['weekday_int','weekday', 'event']).count().sort_values(by=['weekday_int'])

In [ ]:
df1.loc[(df1['weekday'] == 'Saturday') & (df1['event'] == 'Normalday') ,'event'] = "weekend"
df1.loc[(df1['weekday'] == 'Sunday') & (df1['event'] == 'Normalday') ,'event'] = "weekend"

In [ ]:
df1['event'].value_counts()

In [ ]:
df1.groupby(['weekday_int','weekday', 'event']).count().sort_values(by=['weekday_int'])

In [ ]:
df1.head()

In [ ]:
df2.head(3)

In [ ]:
df3.head()

In [ ]:
df2.groupby(['yearweek','store_code', 'category'])['sell_price'].sum()

In [ ]:
sales_melted = df3.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], var_name='d', value_name='value_sales')

# Juntar os datasets com base na coluna 'd'
merged_df = pd.merge(sales_melted, df1[['d', 'date', 'event', 'yearweek']], on='d')

In [ ]:
merged_df.head()

In [ ]:
# Agrupar os valores por regiao e data, fazendo a soma da coluna value_sales e mostrando todas as colunas
sales_agg = merged_df.groupby(['date', 'event', 'yearweek']).agg({
    'value_sales': 'sum',
    'item':'first',	
    'category':'first',		
    'department':'first',		
    'store':'first',		
    'store_code':'first',		
    'region':'first',		
    'd':'first'
}).reset_index()  # Resetando o indice para criar um novo dataframe


In [ ]:
sales_agg.head()

In [ ]:
sales_agg.groupby(['item','category', 'store_code', 'yearweek'])['value_sales'].mean()